In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
# os.makedirs("/content/drive/My Drive/work", exist_ok=True)
%cd "/content/drive/My Drive/work"

In [ ]:
# !git clone https://github.com/artidoro/qlora
%cd qlora
!pip install -U -r requirements.txt

In [ ]:
!huggingface-cli login

In [ ]:
%cd ../

In [ ]:
!git clone https://github.com/facebookresearch/llama

In [ ]:
%cd llama

In [ ]:
!pip install -e .

In [ ]:
%cd ../

In [ ]:
%cd qlora

In [ ]:
model_name="meta-llama/Llama-2-7b-chat-hf"
dataset_name="./test.json"

!python qlora.py \
    --model_name $model_name \
    --output_dir ./output/test\
    --dataset $dataset_name \
    --dataset_format input-output\
    --max_steps 1000 \
    --use_auth \
    --logging_steps 10 \
    --save_strategy steps \
    --data_seed 42 \
    --save_steps 500 \
    --save_total_limit 40 \
    --dataloader_num_workers 1 \
    --group_by_length \
    --logging_strategy steps \
    --remove_unused_columns False \
    --do_train \
    --lora_r 64 \
    --lora_alpha 16 \
    --lora_modules all \
    --double_quant \
    --quant_type nf4 \
    --bits 4 \
    --warmup_ratio 0.03 \
    --lr_scheduler_type constant \
    --gradient_checkpointing \
    --source_max_len 16 \
    --target_max_len 4096 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 16 \
    --eval_steps 187 \
    --learning_rate 0.0002 \
    --adam_beta2 0.999 \
    --max_grad_norm 0.3 \
    --lora_dropout 0.1 \
    --weight_decay 0.0 \
    --seed 0 \
    --load_in_4bit \
    --use_peft \
    --batch_size 4 \
    --gradient_accumulation_steps 2 \
    -- fp16

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

#load base model

model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


#LoRAのパスを指定
peft_name = "./output/test/checkpoint-1000/adapter_model"
model = PeftModel.from_pretrained(
    model,
    peft_name,
    device_map={"":0}
)
model.eval()

device = "cuda:0"
def ask(text):
  inputs = tokenizer(text, return_tensors="pt").to(device)

  with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)
  print(outputs)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

text = "OpenCVってなに？"

ask(text)

In [ ]:
%cd ../

## llama.cpp


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

In [ ]:
%cd llama.cpp

In [ ]:
!make -j && ./main -m ./llama-v2.gguf  -p "Give me the code to get a random number in python." -n 400 -e

#### QLora llama

In [ ]:
!python3 convert-lora-to-ggml.py ./output/test/checkpoint-1000/adapter_model

In [ ]:
!./main -m ./llama-v2.gguf --lora./qlora/output/test/checkpoint-1000/adapter_model/ggml-adapter-model.bin -p "" -n 400 -e
